In [1]:
!pip install "fsspec==2023.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requir

In [2]:
!pip install transformers datasets evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [3]:
import torch
import math
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
def tokenize(example):
    return tokenizer(example["text"])

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
block_size = 128

def group_texts(examples):
    joined = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(joined["input_ids"]) // block_size) * block_size
    result = {
        k: [t[i:i + block_size] for i in range(0, total_length, block_size)]
        for k, t in joined.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_data = tokenized.map(group_texts, batched=True)


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
train_data = lm_data["train"].select(range(8000))
eval_data = lm_data["validation"].select(range(1000))

In [9]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
args = TrainingArguments(
    output_dir="/tmp",
    eval_strategy="epoch",
    learning_rate=3e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    save_strategy="no",
    logging_steps=10,
    push_to_hub=False,
    report_to="none",
)


In [11]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-11-782557086.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.400100,3.404310
2,3.287000,3.397978
3,3.165100,3.401482


TrainOutput(global_step=3000, training_loss=3.3624586849212648, metrics={'train_runtime': 1062.6431, 'train_samples_per_second': 22.585, 'train_steps_per_second': 2.823, 'total_flos': 1567752192000000.0, 'train_loss': 3.3624586849212648, 'epoch': 3.0})

In [12]:
def top_k_accuracy(model, dataset, k=5, n=100):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    correct, total = 0, 0

    for d in dataset.select(range(n)):
        ids = torch.tensor(d["input_ids"]).unsqueeze(0)
        labels = torch.tensor(d["labels"]).unsqueeze(0)
        ids, labels = ids.to(model.device), labels.to(model.device)

        with torch.no_grad():
            logits = model(ids).logits[:, :-1, :]
            topk = logits.topk(k, dim=-1).indices
            match = (topk == labels[:, 1:].unsqueeze(-1)).any(-1)
            correct += match.sum().item()
            total += match.numel()

    return correct / total

topk = top_k_accuracy(model, eval_data, k=5, n=100)
print(f"Top-5 Accuracy: {topk * 100:.2f}%")


Top-5 Accuracy: 62.43%


In [13]:
result = trainer.evaluate()
perplexity = math.exp(result["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

Perplexity: 30.01


In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

prompt = "The indian ocean"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

output = model.generate(
    **inputs,
    max_new_tokens=10,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.5,
    pad_token_id=tokenizer.eos_token_id
)

generated = tokenizer.decode(output[0], skip_special_tokens=True)
print("Prompt:", prompt)
print("Generated text:", generated)

Prompt: The indian ocean
Generated text: The indian ocean is the most abundant in the world . 

